# Imports

In [ ]:
# Suprress warnings
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    # Standard imports
    import matplotlib
    import matplotlib.pyplot as plt
    import matplotlib.patches as patches
    import matplotlib.cm as cm
    import numpy as np

    # Drake imports
    from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
    import pydrake
    from pydrake.all import LogOutput, DirectCollocation, DirectTranscription, MathematicalProgram, InputPortSelection

# Imports of other project files
from log_wrapper import LogWrapper
import constants
import finger
import pedestal
from paper import Paper

from pydrake.all import (MultibodyPlant, Parser, DiagramBuilder, Simulator, RigidTransform,
                         PlanarSceneGraphVisualizer, SceneGraph, TrajectorySource,
                         SnoptSolver, MultibodyPositionToGeometryPose, PiecewisePolynomial,
                         MathematicalProgram, JacobianWrtVariable, eq, RollPitchYaw, AutoDiffXd, BodyIndex)

# Other imports
import importlib

import scipy.interpolate

In [ ]:
# Matplotlib configuring
plt.style.use(['science', 'no-latex'])
font = {'size'   : 14}
matplotlib.rc('font', **font)

In [ ]:
# Meshcat init
proc, zmq_url, web_url = start_zmq_server_as_subprocess()

# Simulation setup
## Pre-finalize steps

In [ ]:
builder = pydrake.systems.framework.DiagramBuilder()

# Add all elements
plant, scene_graph = pydrake.multibody.plant.AddMultibodyPlantSceneGraph(builder, time_step=constants.DT)
v_stiction=1e-3
plant.set_stiction_tolerance(v_stiction)
plant.set_penetration_allowance(0.001)
pedestal_instance = pedestal.AddPedestal(plant)

# These joint angles start the paper approximately the right spot, no matter how many links are used
num_links = 2
# def_joint_angles = list(np.zeros(num_links))
# def_joint_angles[0] = np.pi/100
# def_joint_angles[1] = np.pi/100
# def_joint_angles[int(num_links*0.65)-2] = -np.pi/10
# def_joint_angles[int(num_links*0.65)-1] = -np.pi/10
# def_joint_angles[int(num_links*0.65)] = -np.pi/10
# def_joint_angles[int(num_links*0.65)+1] = -np.pi/10
# # def_joint_angles[int(num_links*0.65)+2] = -np.pi/10
def_joint_angles = 0#np.pi/3

paper = Paper(plant, scene_graph, num_links, "NATURAL", default_joint_angle=def_joint_angles,
              stiffness=0,#2.5e-2,
              damping=0,)#7.12547340446979e-06)
paper.weld_paper_edge(pedestal.PEDESTAL_WIDTH, pedestal.PEDESTAL_HEIGHT)

finger_instance, finger_body, finger_shape = finger.AddFinger(plant, constants.INIT_Y, constants.INIT_Z)

# Set up logger (needs to happen after all bodies are added)
log_wrapper = LogWrapper(plant.num_bodies(), int(finger_body.index()), paper)
builder.AddSystem(log_wrapper)

Calculate link inertia to use in damping calculations:

In [ ]:
paper.plant.get_body(BodyIndex(paper.get_free_edge_idx())).default_rotational_inertia().CalcPrincipalMomentsOfInertia()[0]

## Controller selection

In [ ]:
## CHOOSE CONTROL SYSTEM HERE BY UNCOMMENTING
# # PD control: hits too low
# finger_ctrlr = finger.PDFinger(
#     plant,
#     int(finger_instance),
#     [
#         [constants.INIT_Y, constants.INIT_Z],
#         [constants.INIT_Y*1.1, constants.INIT_Z],
#         [constants.INIT_Y*1.1, pedestal.PEDESTAL_HEIGHT+constants.FINGER_RADIUS/2+0.05],
#         [0, pedestal.PEDESTAL_HEIGHT+constants.FINGER_RADIUS/2+0.05],
#         [0, pedestal.PEDESTAL_HEIGHT+constants.FINGER_RADIUS/2],
#     ],
#     tspan_per_segment=1,
#     ky=10,
#     kz=10
# )

# PD control: hits too high
# finger_ctrlr = finger.PDFinger(
#     plant,
#     int(finger_instance),
#     [
#         [constants.INIT_Y, constants.INIT_Z],
#         [constants.INIT_Y*1.1, constants.INIT_Z],
#         [constants.INIT_Y*1.1, pedestal.PEDESTAL_HEIGHT+constants.FINGER_RADIUS/2+0.08],
#         [0, pedestal.PEDESTAL_HEIGHT+constants.FINGER_RADIUS/2+0.08],
#         [0, pedestal.PEDESTAL_HEIGHT+constants.FINGER_RADIUS/2],
#     ],
#     tspan_per_segment=1,
#     ky=10,
#     kz=10
# )


# Edge feedback
finger_ctrlr = finger.EdgeController(
    plant,
    paper,
    finger_idx=int(finger_instance),
    ll_idx=int(paper.get_free_edge_idx()),
    F_Nd=2,
    debug=True,
)

# # Optimization controller
# finger_ctrlr = finger.OptimizationController(
#     plant,
#     paper,
#     int(finger_instance),
#     paper.get_free_edge_instance()
# )

# Blank controller
# finger_ctrlr = finger.BlankController(
#     plant,
#     int(finger_instance)
# )

## Post-finalize steps

In [ ]:
plant.Finalize()

## Post finalize steps
# Conect finger controller
builder.AddSystem(finger_ctrlr)
builder.Connect(finger_ctrlr.get_output_port(), plant.get_actuation_input_port(finger_instance))
builder.Connect(plant.get_body_poses_output_port(), finger_ctrlr.get_input_port(0))
builder.Connect(plant.get_body_spatial_velocities_output_port(), finger_ctrlr.get_input_port(1))
builder.Connect(plant.get_contact_results_output_port(), finger_ctrlr.get_input_port(2))

# Add logger
builder.Connect(plant.get_body_poses_output_port(), log_wrapper.get_input_port(0))
builder.Connect(plant.get_body_spatial_velocities_output_port(), log_wrapper.get_input_port(1))
builder.Connect(plant.get_body_spatial_accelerations_output_port(), log_wrapper.get_input_port(2)) 
builder.Connect(plant.get_contact_results_output_port(), log_wrapper.get_input_port(3))
builder.Connect(plant.get_reaction_forces_output_port(), log_wrapper.get_input_port(4))

# Visualization and logging
logger = LogOutput(log_wrapper.get_output_port(), builder)
vis = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(builder, scene_graph)
vis.set_planar_viewpoint(camera_position=[1, 0, 0], xmin=-0.3, xmax=0.3, ymin=-0.3, ymax=0.3)

# Build diagram and do actions requiring 
diagram = builder.Build()
diagram_context = diagram.CreateDefaultContext()

In [ ]:
if type(finger_ctrlr) is finger.OptimizationController:
    finger_ctrlr.optimize(plant.GetPositions(diagram_context))

# Run simulation

In [ ]:
# Finalize simulation and visualization
simulator = pydrake.systems.analysis.Simulator(diagram, diagram_context)
simulator.Initialize()
vis.start_recording()
try:
    simulator.AdvanceTo(constants.TSPAN)
except RuntimeError as e:
    print(e)

# This way, we can use the actual time the simulation ran for
effective_tspan = max(logger.sample_times())

vis.stop_recording()
vis.publish_recording()

# Plots used in orginal paper

In [ ]:
# This script is getting huge. We need to turn off parts that we're not using.
generate_orig_plots = True

In [ ]:
if generate_orig_plots:
    # Plot manipulator position vs. trajectory
    # PROGRAMMING: Use body index for finger instead of model instance
    if type(finger_ctrlr) is finger.PDFinger:
        plt.figure(figsize=(2*3,2*2))
        plt.plot(logger.data()[log_wrapper.entries_per_body*int(finger_instance)+1],
                 logger.data()[log_wrapper.entries_per_body*int(finger_instance)+2],
                 label='Manipulator position')
        plt.plot(finger_ctrlr.ys, finger_ctrlr.zs, label='Trajectory')
        plt.xlabel("$y$ position")
        plt.ylabel("$z$ position")
        plt.legend()
        plt.show()

In [ ]:
if generate_orig_plots:
    # Generate plots for paper
    nb = plant.num_bodies()
    y_traces = []
    z_traces = []
    theta_traces = []
    for b in paper.link_idxs:
        y_traces.append(logger.data()[log_wrapper.entries_per_body*b+1])
        z_traces.append(logger.data()[log_wrapper.entries_per_body*b+2])
        theta_traces.append(logger.data()[log_wrapper.entries_per_body*b+3])
    y_traces = np.array(y_traces)
    z_traces = np.array(z_traces)
    theta_traces = np.array(theta_traces)

    times_ = np.arange(0,effective_tspan, effective_tspan/10)
    cmap = cm.get_cmap("viridis_r")
    plt.figure(figsize=(2*3,2*2))
    if type(finger_ctrlr) is finger.PDFinger:
        plt.plot(finger_ctrlr.ys, finger_ctrlr.zs, '--k', zorder=-1)
    for t in times_:
        c = cmap(t/effective_tspan)
        idx = np.argmax(logger.sample_times() >= t)

        # Plot paper
        # PROGRAMMING: Account for paper thickness in plots
        for y, z, theta in zip(y_traces[:,idx], z_traces[:,idx], theta_traces[:,idx]):
            y0 = y - np.cos(theta)*paper.link_width/2
            z0 = z - np.sin(theta)*paper.link_width/2
            y1 = y + np.cos(theta)*paper.link_width/2
            z1 = z + np.sin(theta)*paper.link_width/2
            plt.plot([y0, y1], [z0, z1], color=c)


        # Plot manipulator
        plt.scatter(logger.data()[log_wrapper.entries_per_body*int(finger_instance)+1,idx], 
                    logger.data()[log_wrapper.entries_per_body*int(finger_instance)+2,idx],
                    color=c, s=300, zorder=1)

    xlim = plt.xlim()
    ylim = plt.ylim()
    plt.scatter([xlim[0]-50, xlim[0]-50], [ylim[0]-50, ylim[0]-50], c=[0, effective_tspan], cmap=cmap)
    plt.xlim(xlim)
    plt.ylim(ylim)
    cb = plt.colorbar()
    cb.set_label("Time")
    plt.xlabel("$y$ position")
    plt.ylabel("$z$ position")
    plt.show()

# Drag plots

This is a sanity check to see how much force we're neglecting by ignoring drag. The colored lines are the drag forces at each link, and the dashed black line is the force due to gravity for scale.

Note that the formula for drag force is $\rho v^2 C_d A_{paper}$.

$C_d$ comes from [this site](https://www.engineersedge.com/fluid_flow/rectangular_flat_plate_drag_14036.htm), which says it depends on $L/d$. For 20 links, $L/d\approx15$. Rounding this down to 10, we get $C_d=1.22$.

In [ ]:
generate_drag_plots = False

In [ ]:
if generate_drag_plots:
    # Plot drag forces
    plt.figure(figsize=(16, 12))
    nb = plant.num_bodies
    vel_traces = []
    for b in paper.link_idxs:
        rho = 1.225
        C_d = paper.link_width*paper.depth
        vel_sqred = logger.data()[log_wrapper.entries_per_body*b+7]**2+logger.data()[log_wrapper.entries_per_body*b+8]**2
        f = 0.5*rho*C_d*vel_sqred
        plt.plot(f)

    plt.axhline(paper.link_mass*9.81, color='black', linestyle='--')
    plt.show()

# Verifying equations hold
We have the following free body diagrams:
<img src="forces.png" width=400px>
<img src="link-moments.png" width=400px>

Which give us the following equations:
$$
\begin{aligned}
     m_La_{LT} &= F_{FL} + F_{GT} +F_{OT} \\
    m_La_{LN} &= F_{NL} + F_{GN} +F_{ON} \\
    m_Ma_{MT} &= F_{FM} + F_{CT} \\
    m_Ma_{MN} &= F_{NM} + F_{CN} \\
    a_{LT} &= a_{MT} \\
    a_{LN} &= a_{MN} \\
    a_{LT} &= -\frac{w_L}{2}\dot\theta^2 \\
    a_{LN} &= \frac{w_L}{2}\ddot\theta \\
    I_L\ddot\theta &= \frac{w_L}{2}F_{ON}- \frac{h_L}{2}F_{FL} - r_TF_{NL} \\
    F_{NL} &= -F_{NM} \\
    F_{FL} &= -F_{FM} \\
    a_{NL} &= a_{Nd}  \\
    F_{FL} &= 0
\end{aligned}
$$
For each of these equations, I want to plot the value on the left side and the value on the right side to see if they match.

## Loading signals
Preparing all the signals we'll need.

In [ ]:
debug = finger_ctrlr.debug

In [ ]:
l_times = logger.sample_times()
d_times = np.array(debug['times'])

### Helper functions

In [ ]:
def get_N_proj(vec):
    N_vec = np.matmul(N_hat_proj_mat, vec)
    N_mag = np.linalg.norm(N_vec, axis=1)
    N_sgn = np.sign(np.matmul(np.transpose(N_hats, [0, 2, 1]), N_vec))
    N = N_mag.flatten()*N_sgn.flatten()
    return N

In [ ]:
def get_T_proj(vec):
    T_vec = np.matmul(T_hat_proj_mat, vec)
    T_mag = np.linalg.norm(T_vec, axis=1)
    T_sgn = np.sign(np.matmul(np.transpose(T_hats, [0, 2, 1]), T_vec))
    T = T_mag.flatten()*T_sgn.flatten()
    return T

In [ ]:
def diff_vec(vec):
    d_vec = np.diff(vec, axis=0)
    for i in range(vec.shape[1]):
        d_vec[:, i] /= np.expand_dims(np.diff(l_times), 1)
    d_vec = np.concatenate((d_vec, [d_vec[-1]]))
    return d_vec

In [ ]:
def plot_vec(sig1, sig1_label, sig2, sig2_label, finish_plotting=True, plot_x=True):
    plt.figure(figsize=(16, 8))
    lw = 5
    if plot_x:
        plt.plot(l_times, sig1[:,0], label=sig1_label + r' $\hat x$ component', color='pink', linewidth=lw)
    plt.plot(l_times, sig1[:,1], label=sig1_label + r' $\hat y$ component', color='lightgreen', linewidth=lw)
    plt.plot(l_times, sig1[:,2], label=sig1_label + r' $\hat z$ component', color='lightskyblue', linewidth=lw)
    
    if plot_x:
        plt.plot(l_times, sig2[:,0], label=sig2_label + r' $\hat x$ component', color='red', linewidth=lw, linestyle='--')
    plt.plot(l_times, sig2[:,1], label=sig2_label + r' $\hat y$ component', color='green', linewidth=lw, linestyle='--')
    plt.plot(l_times, sig2[:,2], label=sig2_label + r' $\hat z$ component', color='blue', linewidth=lw, linestyle='--')
    plt.xlabel("Time (seconds)")
    plt.legend()
    plt.autoscale(enable=True, axis='x', tight=True)
    if finish_plotting:
        plt.show()

In [ ]:
def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'same') / w

In [ ]:
def moving_average_vec(x, w):
    out = np.zeros_like(x)
    for i in range(x.shape[1]):
        out[:,i] = np.expand_dims(moving_average(x[:,i].flatten(), w), 1)
    return out

### Inputs
These should all come from simulation/geometry, since they are inputs to the controller and are "fixed" from the controller's perspective.
#### Geometric quantities ($m_L, m_M, w_L, I_L, h_L, \mu$)

In [ ]:
m_L = paper.link_mass
m_M = constants.FINGER_MASS
w_L = paper.link_width
I_L = paper.plant.get_body(
    BodyIndex(paper.get_free_edge_idx())).default_rotational_inertia().CalcPrincipalMomentsOfInertia()[0]
h_L = paper.height
mu = constants.FRICTION
r = constants.FINGER_RADIUS

#### Vectors which aren't determined by the force balance ($F_{GT}, F_{GN} \hat T, \hat N$)
- [ ] PROGRAMMING: also get $\hat T$ from simulation somehow

In [ ]:
N_hats_raw = logger.data()[log_wrapper.contact_entry_start_idx+14:log_wrapper.contact_entry_start_idx+17,:]
N_hats = np.expand_dims(N_hats_raw.T, 2)

In [ ]:
T_hats_no_interp = np.array(debug['T_hats'])
T_hats = scipy.interpolate.interp1d(d_times, T_hats_no_interp, axis=0)(l_times)
T_hats.shape

It's also useful here to calculate project matrices:

In [ ]:
T_hat_proj_mat = np.matmul(T_hats, np.transpose(T_hats, [0, 2, 1]))

In [ ]:
N_hat_proj_mat = np.matmul(N_hats, np.transpose(N_hats, [0, 2, 1]))

- [ ] PROGRAMMING: Calc these directly instead
- [ ] PROGRAMMING: Stop interpolating everything (avoid using `debug`)

In [ ]:
F_GT_no_interp = np.array(debug['F_GTs'])
F_GT = scipy.interpolate.interp1d(d_times, F_GT_no_interp, axis=0)(l_times)
F_GN_no_interp = np.array(debug['F_GNs'])
F_GN = scipy.interpolate.interp1d(d_times, F_GN_no_interp, axis=0)(l_times)

#### Positions and velocities which are "fixed'' until the next timestep ($\dot\theta^2, r_T, \dot d_T$)

##### Poses and velocities of objects

In [ ]:
d_theta = logger.data()[log_wrapper.entries_per_body*paper.get_free_edge_idx() + 6 + 3]

Note that $r_T$ here is calculated differently (but more accurately) than in the controller.

In [ ]:
p_contact = np.expand_dims(
    logger.data()[log_wrapper.contact_entry_start_idx+11:log_wrapper.contact_entry_start_idx+14,:].T,
    2)

In [ ]:
p_link = np.expand_dims(
    logger.data()[log_wrapper.entries_per_body*paper.get_free_edge_idx():
                  log_wrapper.entries_per_body*paper.get_free_edge_idx()+3].T,
    2)
# Useful alias
p_L = p_link

In [ ]:
p_LN = get_N_proj(p_L)
p_LT = get_T_proj(p_L)

In [ ]:
v_L = np.expand_dims(
    logger.data()[log_wrapper.entries_per_body*paper.get_free_edge_idx()+6:
                  log_wrapper.entries_per_body*paper.get_free_edge_idx()+6+3].T,
    2)

In [ ]:
v_LN = get_N_proj(v_L)
v_LT = get_T_proj(v_L)

In [ ]:
p_M = np.expand_dims(logger.data()[log_wrapper.entries_per_body*int(finger_body.index()):
                    log_wrapper.entries_per_body*int(finger_body.index())+3,:].T, 2)

In [ ]:
p_MN = get_N_proj(p_M)
p_MT = get_T_proj(p_M)

In [ ]:
v_M = np.expand_dims(
    logger.data()[log_wrapper.entries_per_body*int(finger_body.index())+6:
                  log_wrapper.entries_per_body*int(finger_body.index())+6+3].T,
    2)

In [ ]:
v_MN = get_N_proj(v_M)
v_MT = get_T_proj(v_M)

##### Derived points and velocities

In [ ]:
p_link_contact = p_contact - p_link
p_link_contact_T = get_T_proj(p_link_contact)
r_T = p_link_contact_T

In [ ]:
p_LE = p_link+ (w_L/2)*T_hats-(h_L/2)*N_hats

In [ ]:
p_LEN = get_N_proj(p_LE)
p_LET = get_T_proj(p_LE)

In [ ]:
# PROGRAMMING: get this from contact point instead

In [ ]:
d_vec = p_contact - p_LE
d_T = get_T_proj(d_vec)
d_N = get_N_proj(d_vec)

In [ ]:
d_d_vec = diff_vec(d_vec)
d_d_T = get_T_proj(d_d_vec)
d_d_N = get_N_proj(d_d_vec)

In [ ]:
v_LE = diff_vec(p_LE)

In [ ]:
v_LEN = get_N_proj(v_LE)
v_LET = get_T_proj(v_LE)

In [ ]:
v_contact = diff_vec(p_contact)
a_contact = diff_vec(v_contact)

In [ ]:
p_C = p_contact
v_C = v_contact
a_C = a_contact

In [ ]:
p_E = p_LE
v_E = diff_vec(p_E)
a_E = diff_vec(v_E)

###### Supporting plots

In [ ]:
d_p_M = diff_vec(p_M)

In [ ]:
d_v_M = diff_vec(v_M)

In [ ]:
plt.figure(figsize=(16, 8))
lw = 5
plt.subplot(211)
plt.plot(l_times, d_T, label='$\hat T$ component', linewidth=lw)
plt.xlabel("Time (seconds)")
plt.legend()
plt.subplot(212)
plt.plot(l_times, d_N, label='$\hat N$ component', linewidth=lw)
plt.xlabel("Time (seconds)")
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(16, 8))
lw = 5
plt.subplot(211)
plt.plot(l_times, d_d_T, label='$\hat T$ component', linewidth=lw)
plt.xlabel("Time (seconds)")
plt.legend()
plt.subplot(212)
plt.plot(l_times, d_d_N, label='$\hat N$ component', linewidth=lw)
plt.xlabel("Time (seconds)")
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(16, 8))
lw = 5
plt.plot(l_times, d_p_M[:,0,:], label='$\dot p_{Mx}$', color='pink', linewidth=lw)
plt.plot(l_times, d_p_M[:,1,:], label='$\dot p_{My}$', color='lightgreen', linewidth=lw)
plt.plot(l_times, d_p_M[:,2,:], label='$\dot p_{Mz}$', color='lightskyblue', linewidth=lw)
plt.plot(l_times, v_M[:,0,:], '--', label='$v_{Mx}$', color='red', linewidth=lw)
plt.plot(l_times, v_M[:,1,:], '--', label='$v_{My}$', color='green', linewidth=lw)
plt.plot(l_times, v_M[:,2,:], '--', label='$v_{Mz}$', color='blue', linewidth=lw)
plt.xlabel("Time (seconds)")
plt.legend()
plt.show()

#### Control inputs ($a_{Nd}$)
- [ ] PROGRAMMING: Pass in `a_Nd` instead of `F_Nd`

In [ ]:
a_Nd = finger_ctrlr.F_Nd/paper.link_mass

In [ ]:
bar_d_T = d_T

### Outputs
These should all come directly from simulation, except for control forces.
#### Accelerations of both objects ($a_{LT}, a_{LN}, a_{MT}, a_{MN}, \ddot\theta$)

In [ ]:
a_link_raw = logger.data()[log_wrapper.entries_per_body*paper.link_idxs[-1] + 12:
                     log_wrapper.entries_per_body*paper.link_idxs[-1]+12+3,:]
a_link = np.expand_dims(a_link_raw.T, 2)

In [ ]:
a_LT_vec = np.matmul(T_hat_proj_mat, a_link)
a_LT_mag = np.linalg.norm(a_LT_vec, axis=1)
a_LT_sgn = np.sign(np.matmul(
    np.transpose(T_hats, [0, 2, 1]),
    a_LT_vec))
a_LT = a_LT_mag.flatten()*a_LT_sgn.flatten()

In [ ]:
a_LN_vec = np.matmul(N_hat_proj_mat, a_link)
a_LN_mag = np.linalg.norm(a_LN_vec, axis=1)
a_LN_sgn = np.sign(np.matmul(
    np.transpose(N_hats, [0, 2, 1]),
    a_LN_vec))
a_LN = a_LN_mag.flatten()*a_LN_sgn.flatten()

In [ ]:
a_man_raw = logger.data()[log_wrapper.entries_per_body*int(finger_body.index()) + 12:
                     log_wrapper.entries_per_body*int(finger_body.index())+12+3,:]
a_man = np.expand_dims(a_man_raw.T, 2)

In [ ]:
a_MT_vec = np.matmul(T_hat_proj_mat, a_man)
a_MT_mag = np.linalg.norm(a_MT_vec, axis=1)
a_MT_sgn = np.sign(np.matmul(
    np.transpose(T_hats, [0, 2, 1]),
    a_MT_vec))
a_MT = a_MT_mag.flatten()*a_MT_sgn.flatten()

In [ ]:
a_MN_vec = np.matmul(N_hat_proj_mat, a_man)
a_MN_mag = np.linalg.norm(a_MN_vec, axis=1)
a_MN_sgn = np.sign(np.matmul(
    np.transpose(N_hats, [0, 2, 1]),
    a_MN_vec))
a_MN = a_MN_mag.flatten()*a_MN_sgn.flatten()

In [ ]:
dd_theta = logger.data()[log_wrapper.entries_per_body*paper.get_free_edge_idx() + 12 + 3,:]

#### $\ddot d_T$

In [ ]:
dd_d_vec = diff_vec(d_d_vec)
dd_d_T = get_T_proj(dd_d_vec)
dd_d_N = get_N_proj(dd_d_vec)

##### Plots to check signals 

In [ ]:
plt.figure(figsize=(16, 8))
lw = 5
plt.subplot(211)
plt.plot(l_times, dd_d_T, label='$\hat T$ component', linewidth=lw)
plt.xlabel("Time (seconds)")
plt.legend()
plt.subplot(212)
plt.plot(l_times, dd_d_N, label='$\hat N$ component', linewidth=lw)
plt.xlabel("Time (seconds)")
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(16, 8))
lw = 5
plt.plot(l_times, d_v_M[:,0,:], label='$\dot v_{Mx}$', color='pink', linewidth=lw)
plt.plot(l_times, d_v_M[:,1,:], label='$\dot v_{My}$', color='lightgreen', linewidth=lw)
plt.plot(l_times, d_v_M[:,2,:], label='$\dot v_{Mz}$', color='lightskyblue', linewidth=lw)
plt.plot(l_times, a_man[:,0,:], '--', label='$a_{Mx}$', color='red', linewidth=lw)
plt.plot(l_times, a_man[:,1,:], '--', label='$a_{My}$', color='green', linewidth=lw)
plt.plot(l_times, a_man[:,2,:], '--', label='$a_{Mz}$', color='blue', linewidth=lw)
plt.xlabel("Time (seconds)")
plt.ylim(-1e3, 1e3)
plt.legend()
plt.show()

#### Contact, object, and control forces on each object ($F_{NL}, F_{FL}, F_{ON}, F_{OT}, F_{NM}, F_{FM}, F_{CT}, F_{CN}$

##### Contact

In [ ]:
F_contact_M_raw = logger.data()[log_wrapper.contact_entry_start_idx+0:log_wrapper.contact_entry_start_idx+3,:]
F_contact_M = np.expand_dims(F_contact_M_raw.T, 2)

In [ ]:
F_NM_vec = np.matmul(N_hat_proj_mat, F_contact_M)
F_FM_vec = F_contact_M - F_NM_vec

In [ ]:
F_NM_mag = np.linalg.norm(F_NM_vec, axis=1)
F_NM_sgn = np.sign(np.matmul(
    np.transpose(N_hats, [0, 2, 1]),
    F_NM_vec))
F_NM = F_NM_mag.flatten()*F_NM_sgn.flatten()

In [ ]:
F_FM_mag = np.linalg.norm(F_FM_vec, axis=1)
F_FM_sgn = np.sign(np.matmul(
    np.transpose(T_hats, [0, 2, 1]),
    F_FM_vec))
F_FM = F_FM_mag.flatten()*F_FM_sgn.flatten()

In [ ]:
F_FL = -F_FM
F_NL = -F_NM

##### Object

In [ ]:
joint_force_in_compliance_frame = logger.data()[
    log_wrapper.joint_entry_start_idx:log_wrapper.joint_entry_start_idx+3,:]

In [ ]:
F_OT = joint_force_in_compliance_frame[1,:]
F_ON = joint_force_in_compliance_frame[2,:]

##### Control

In [ ]:
F_CT_no_interp = np.array(debug['F_CTs'])
F_CT = scipy.interpolate.interp1d(d_times, F_CT_no_interp, axis=0)(l_times)
F_CN_no_interp = np.array(debug['F_CNs'])
F_CN = scipy.interpolate.interp1d(d_times, F_CN_no_interp, axis=0)(l_times)

## Plotting settings

In [ ]:
lw = 5

In [ ]:
figsize = (16, 8)

In [ ]:
plot_supporting_plots = True

In [ ]:
t_start = 0.01
t_end = 0.065
idx_start = np.argmax(l_times > t_start)
idx_end = np.argmax(l_times > t_end)

In [ ]:
idx_lose_contact = np.argmax(np.isnan(F_FL[idx_start:])) + idx_start
slip_speed = logger.data()[log_wrapper.contact_entry_start_idx+10,:]
t_lose_contact = t_slip = l_times[np.argmax(slip_speed > v_stiction)] #l_times[idx_lose_contact]
t_lose_contact

In [ ]:
def plot_eq(lhs, lhs_label, rhs, rhs_label, lhs_alpha=1, rhs_alpha=1, lw=5, finish_plotting=True, plot_diff=True, figsize_=None):
    if figsize_ is not None:
        plt.figure(figsize=figsize_)
    else:
        plt.figure(figsize=figsize)
    if isinstance(lhs, float) or isinstance(lhs, int):
        lhs = np.ones_like(l_times)*lhs
    if isinstance(rhs, float) or isinstance(rhs, int):
        rhs = np.ones_like(l_times)*rhs
        
    b = np.broadcast(lhs, rhs)
    if np.count_nonzero(np.array(b.shape) > 1000) > 1:
        raise ValueError("Broadcast shape of lhs and rhs is {}, which is too large.".format(b.shape))
        
    plt.subplot(211)
    plt.plot(l_times[idx_start:idx_end],
            lhs[idx_start:idx_end]-rhs[idx_start:idx_end],
            linewidth=lw)
    plt.autoscale(enable=True, axis='x', tight=True)
    plt.xlabel("Time (s)")
    plt.title("Difference between {} and {}".format(lhs_label, rhs_label))
    
    
    plt.subplot(212)
    plt.plot(l_times[idx_start:idx_end],
             lhs[idx_start:idx_end],
             label=lhs_label, linewidth=lw, color='lightskyblue', zorder=0, alpha=lhs_alpha)
    plt.plot(l_times[idx_start:idx_end],
             rhs[idx_start:idx_end],
             label=rhs_label, linewidth=lw, color='b', linestyle='--', zorder=1, alpha=rhs_alpha)
    plt.autoscale(enable=True, axis='x', tight=True)
    
    
    plt.axvline(0.05, color='green', linewidth=lw)
    plt.axvline(t_lose_contact, color='red', linewidth=lw)
    
    plt.legend()
    plt.xlabel("Time (s)")
    
    if finish_plotting:
        plt.show()

## Plots

### $m_La_{LT} = F_{FL} + F_{GT} +F_{OT}$
*If this plot is correct:*
The free body diagram is correct in thet $\hat T$ direction on the link.

In [ ]:
lhs_label = r'$m_La_{LT}$'
rhs_label = r'$F_{FL} + F_{GT} +F_{OT}$'

lhs = m_L*a_LT

rhs = F_FL+F_GT+F_OT

plot_eq(lhs, lhs_label, rhs, rhs_label)

### $m_La_{LN} = F_{NL} + F_{GN} +F_{ON}$
*If this plot is correct:*
The free body diagram is correct in thet $\hat N$ direction on the link.

In [ ]:
lhs_label = r'$m_La_{LN}$'
rhs_label = r'$F_{NL} + F_{GN} +F_{ON}$'

lhs = m_L*a_LN

rhs = F_NL + F_GN + F_ON

plot_eq(lhs, lhs_label, rhs, rhs_label)

### $m_Ma_{MT} = F_{FM} + F_{CT}$
*If this plot is correct:*
The free body diagram is correct in thet $\hat T$ direction on the manipulator.

In [ ]:
lhs_label = r'$m_Ma_{MT}$'
rhs_label = r'$F_{FM} + F_{CT}$'

lhs = m_M*a_MT

rhs = F_FM + F_CT

plot_eq(lhs, lhs_label, rhs, rhs_label)

### $m_Ma_{MN} = F_{NM} + F_{CN}$
*If this plot is correct:*
The free body diagram is correct in thet $\hat TN direction on the manipulator.

In [ ]:
lhs_label = r'$m_Ma_{MN}$'
rhs_label = r'$ F_{NM} + F_{CN}$'

lhs = m_M*a_MN

rhs = F_NM+F_CN

plot_eq(lhs, lhs_label, rhs, rhs_label)

### $\bar d_{T}\ddot\theta +2\dot d_T\dot\theta= a_{MN} - a_{LN} - \frac{w_L}{2}\ddot\theta - \left(r+ \frac{h_L}{2}\right)\dot\theta^2$

In [ ]:
lhs_label = r'$\bar d_{T}\ddot\theta +2\dot d_T\dot\theta$'
rhs_label = r'$a_{MN} - a_{LN} - \frac{w_L}{2}\ddot\theta - \left(r+ \frac{h_L}{2}\right)\dot\theta^2$'

lhs = bar_d_T*dd_theta+2*d_d_T*d_theta

rhs = a_MN-a_LN-w_L*dd_theta/2-(r+h_L/2)*d_theta**2

plot_eq(lhs, lhs_label, rhs, rhs_label)
plt.show()

### $ \ddot d_T - \bar d_{T} \dot\theta^2  = a_{MT} - a_{LT} - \left(r+ \frac{h_L}{2}\right)\ddot\theta + \frac{w_L}{2}\dot\theta^2$

In [ ]:
lhs_label = r'$\bar d_{T} \dot\theta^2+\ddot d_T$'
rhs_label = r'$a_{MT} - a_{LT} - \left(r+ \frac{h_L}{2}\right)\ddot\theta + \frac{w_L}{2}\dot\theta^2$'

lhs = -d_T*d_theta**2+dd_d_T

rhs = a_MT-a_LT-(r+h_L/2)*dd_theta+(w_L/2)*d_theta**2

plot_eq(lhs, lhs_label, rhs, rhs_label, finish_plotting=False)

plt.legend()
plt.show()

### $a_{LT} = -\frac{w_L}{2}\dot\theta^2$

In [ ]:
lhs_label = r'$a_{LT}$'
rhs_label = r'$-\frac{w_L}{2}\dot\theta^2$'

lhs = a_LT

rhs = -(w_L/2)*d_theta**2

plot_eq(lhs, lhs_label, rhs, rhs_label)

### $a_{LN} = \frac{w_L}{2}\ddot\theta$

In [ ]:
lhs_label = r'$a_{LN}$'
rhs_label = r'$\frac{w_L}{2}\ddot\theta$'

lhs = a_LN

rhs = dd_theta*w_L/2

plot_eq(lhs, lhs_label, rhs, rhs_label)

### $I_L\ddot\theta = -\frac{w_L}{2}F_{ON}+ \frac{h_L}{2}F_{FL} + r_TF_{NL}$

In [ ]:
lhs_label = r'$I_L\ddot\theta$'
rhs_label = r'$-\frac{w_L}{2}F_{ON}+ \frac{h_L}{2}F_{FL} + r_TF_{NL}$'

lhs = I_L*dd_theta

rhs = -F_ON*w_L/2+h_L*F_FL/2+r_T*F_NL

plot_eq(lhs, lhs_label, rhs, rhs_label)

### $F_{NL} = -F_{NM}$

In [ ]:
lhs_label = r'$F_{NL}$'
rhs_label = r'$-F_{NM}$'

lhs = F_NL

rhs = -F_NM

plot_eq(lhs, lhs_label, rhs, rhs_label)

### $F_{FL} = -F_{FM}$

In [ ]:
lhs_label = r'$F_{FL}$'
rhs_label = r'$-F_{FM}$'

lhs = F_FL

rhs = -F_FM

plot_eq(lhs, lhs_label, rhs, rhs_label, rhs_alpha=0.5)

### $a_{LN} = a_{Nd}$

In [ ]:
lhs_label = r'$a_{LN}$'
rhs_label = r'$a_{Nd}$'

lhs = a_LN

rhs = a_Nd

plot_eq(lhs, lhs_label, rhs, rhs_label)

### $F_{FL} = 0$

In [ ]:
lhs_label = r'$F_{FL}$'
rhs_label = r'0'

lhs = F_FL

rhs = 0

plot_eq(lhs, lhs_label, rhs, rhs_label)

# Debugging $\vec d_T$ plots

### Loading signals

In [ ]:
pen_depth = logger.data()[log_wrapper.contact_entry_start_idx+17]

## $\hat T$ component of $\vec d_T$

In [ ]:
lhs_label = r'$d_T$'
rhs_label = r'$p_{MT}-p_{LT}+w_L/2$'
lhs = d_T
rhs = p_MT-p_LT - (w_L)/2

plot_eq(lhs, lhs_label, rhs, rhs_label)

## $\hat N$ component of $\vec d_T$

In [ ]:
lhs_label = r'$0$'
rhs_label = r'$p_{MN}-p_{LN}+r+h_l/2$'
lhs = 0
rhs = p_MN-(p_LN - (h_L)/2)+r

plot_eq(lhs, lhs_label, rhs, rhs_label, finish_plotting=False)
ylims = plt.ylim()
plt.plot(l_times, d_N, linewidth=5, 
         label=r'Measured $\hat N$ component of $\vec d$', 
         color='purple', zorder=0)
plt.plot(l_times, d_N+pen_depth/2, linewidth=5, 
         label=r'Measured $\hat N$ component of $\vec d$, plus penetration_depth/2', 
         color='orange', zorder=0)
plt.ylim(ylims)
plt.legend()
plt.show()

These differ by penetration depth.

## $\hat T$ component of $\frac{d}{dt}\left(\vec d_T\right)$

In [ ]:
lhs_label = r'$\dot d_T$'
rhs_label = r'$v_{MT}-v_{LT}-\frac{\dot\theta}{2}\left(2r + h_L\right)$'
lhs = d_d_T
rhs = v_MT-v_LT-d_theta*(2*r+h_L)/2

plot_eq(lhs, lhs_label, rhs, rhs_label, finish_plotting=False)

## $\hat N$ component of $\frac{d}{dt}\left(\vec d_T\right)$

In [ ]:
lhs_label = r'$d_T\dot\theta$'
rhs_label = r'$v_{MT}-v_{LT}-\frac{\dot\theta}{2}\left(2r + h_L\right)$'
lhs = d_T*d_theta
rhs = v_MN-v_LN-d_theta*w_L/2

plot_eq(lhs, lhs_label, rhs, rhs_label, finish_plotting=False)

## $\hat T$ component of $\frac{d^2}{dt^2}\left(\vec d_T\right)$

In [ ]:
lhs_label = r'$\ddot d_T-d_T\dot\theta^2$'
rhs_label = r'$a_{MT}-a_{LT}-\left(r+h_L/2\right)\ddot\theta+w_l\dot\theta^2/2$'
lhs = dd_d_T-d_T*d_theta**2
rhs = a_MT-a_LT-dd_theta*(2*r+h_L)/2+(d_theta**2)*(w_L)/2

plot_eq(lhs, lhs_label, rhs, rhs_label, finish_plotting=False)
ylims = plt.ylim()
plt.plot(l_times, moving_average(dd_d_T, 100), linewidth=5, 
         label=r'Measured $\hat T$ component of $\ddot{\vec d}$', 
         color='green', zorder=0)
plt.ylim(ylims)
plt.legend()
plt.show()

## $\hat N$ component of $\frac{d^2}{dt^2}\left(\vec d_T\right)$

In [ ]:
lhs_label = r'$2\dot d_T\dot\theta+d_T\ddot\theta$'
rhs_label = r'$a_{MN}-a_{LN}-\left(r+h_L/2\right)\ddot\theta+w_l\dot\theta^2/2$'
lhs = 2*d_d_T*d_theta + d_T*dd_theta
rhs = a_MN-a_LN-dd_theta*w_L/2-(d_theta**2)*(r+h_L/2)

plot_eq(lhs, lhs_label, rhs, rhs_label, finish_plotting=False)
ylims = plt.ylim()
plt.plot(l_times, moving_average(dd_d_N, 100), 
         label=r'Measured $\hat N$ component of $\ddot{\vec d}$', 
         color='green', linewidth=5, zorder=0)
plt.ylim(ylims)
plt.legend()
plt.show()

# What's slip speed?
Supposedly, `speed between body A and B at contact point C.`

In [ ]:
omega_L = d_theta
omega_M = logger.data()[log_wrapper.entries_per_body*int(finger_body.index()) + 6 + 3]

In [ ]:
omega_vec_L = np.expand_dims(np.vstack((omega_L, np.zeros_like(omega_L), np.zeros_like(omega_L))).T, 2)
omega_vec_M = np.expand_dims(np.vstack((omega_M, np.zeros_like(omega_M), np.zeros_like(omega_M))).T, 2)

In [ ]:
v_C_M = v_M + np.cross(omega_vec_M, p_C-p_M, axis=1)

In [ ]:
v_C_L = v_L + np.cross(omega_vec_L, p_C-p_L, axis=1)

In [ ]:
v_C_M_T = get_T_proj(v_C_M)
v_C_L_T = get_T_proj(v_C_L)

In [ ]:
plt.figure(figsize=(16,8))
ylims = plt.ylim()
plt.plot(l_times, v_C_M_T, linewidth=5)
plt.plot(l_times, v_C_L_T, linewidth=5)
plt.plot(l_times, v_C_L_T-v_C_M_T, linewidth=5)
plt.autoscale(enable=True, axis='x', tight=True)
plt.ylim(-1, 1)
plt.show()

In [ ]:
plt.figure(figsize=(16,8))
ylims = plt.ylim()
plt.plot(l_times, np.abs(v_C_M_T-v_C_L_T)-slip_speed, linewidth=5, color='green')
plt.autoscale(enable=True, axis='x', tight=True)
plt.ylim(-1, 1)
plt.show()

## Change in slip speed

In [ ]:
a_s = np.diff(slip_speed)/np.diff(l_times)
a_s = np.concatenate((a_s, [a_s[-1]]))
d_theta_L = d_theta
d_theta_M = omega_M
v_WConM = v_M + np.cross(omega_vec_M, p_C-p_M, axis=1)
v_WConMN = get_N_proj(v_WConM)
v_WConL = v_L + np.cross(omega_vec_L, p_C-p_L, axis=1)
v_WConLN = get_N_proj(v_WConL)

p_LConLN = get_N_proj(p_C - p_L)
p_MConMN = get_N_proj(p_C - p_M)

In [ ]:
dd_theta_L = dd_theta
dd_theta_M = logger.data()[log_wrapper.entries_per_body*int(finger_body.index()) + 12 + 3,:]

In [ ]:
lhs = a_s
lhs_label = r"$a_s$"
rhs = d_theta_L*(2*v_WConLN-v_WConMN-v_LN)
rhs += a_LT
rhs += dd_theta_L*p_LConLN
rhs -= a_MT
rhs += dd_theta_M*p_MConMN
rhs -= d_theta_M*(v_WConMN-v_MN)
rhs_label = r"$\dot\theta_L" +  \
    r"\left(2v_{WConL, N}-v_{WConM, N}- v_{LN}\right)" + \
    r"+" + \
    r"a_{LT}" + \
    r"+" + \
    r"\ddot\theta_L p_{LConL, N}" +  \
    r"-" + \
    r"a_{MT}" + \
    r"-" + \
    r"\ddot\theta_M p_{MConM, N}" + \
    r"-" + \
    r"\dot\theta_M\left(v_{WConM, N} - v_{MN}\right)$"

plot_eq(lhs, lhs_label, rhs, rhs_label)

# Other useful plots

In [ ]:
plt.figure(figsize=(16, 8))
lw = 5
plt.plot(d_times, debug['F_CTs'], label='$F_{CT}$', linewidth=lw)
plt.plot(d_times, debug['F_CNs'], label='$F_{CN}$',linewidth=lw)
plt.xlabel("Time (seconds)")
plt.ylim(-1,1)
plt.axvline(0.1)
plt.legend()
plt.show()

In [ ]:
## Plotting normal for

## Verifying we have the same $\hat N$

In [ ]:
N_hats_sim = N_hats

In [ ]:
N_hats_ctrl = np.array(debug['N_hats'])

In [ ]:
plt.figure(figsize=(16, 8))
lw = 5
plt.plot(l_times, N_hats_sim[:,0,:], label='$\hat n_{BA_W,x}$', color='pink', linewidth=lw)
plt.plot(l_times, N_hats_sim[:,1,:], label='$\hat n_{BA_W,y}$', color='lightgreen', linewidth=lw)
plt.plot(l_times, N_hats_sim[:,2,:], label='$\hat n_{BA_W,z}$', color='lightskyblue', linewidth=lw)
plt.plot(d_times, N_hats_ctrl[:,0,:], '--', label='$\hat N_x$', color='red', linewidth=lw)
plt.plot(d_times, N_hats_ctrl[:,1,:], '--', label='$\hat N_y$', color='green', linewidth=lw)
plt.plot(d_times, N_hats_ctrl[:,2,:], '--', label='$\hat N_z$', color='blue', linewidth=lw)
plt.xlabel("Time (seconds)")
plt.legend()
plt.show()

The signals line up, so our calculation of $\hat N$ matches what Drake gets.

## Does penetration depth make a big difference?
In other words, does $\vec p_{C} + \epsilon \hat N = \vec p_{M} + r\hat N$ match up, and what's the magnitude of $\epsilon$?

In [ ]:
epsilon_vec = np.expand_dims(pen_depth/2, [1, 2])*N_hats

In [ ]:
d_epsilon_vec = diff_vec(epsilon_vec)
d_epsilon_vec_N = get_N_proj(d_epsilon_vec)
d_epsilon_vec_T = get_T_proj(d_epsilon_vec)
dd_epsilon_vec = diff_vec(d_epsilon_vec)
dd_epsilon_vec_N = get_N_proj(dd_epsilon_vec)
dd_epsilon_vec_T = get_T_proj(dd_epsilon_vec)

In [ ]:
dd_d_T_ = dd_d_T - dd_epsilon_vec_T
dd_d_N_ = dd_d_N - dd_epsilon_vec_N
d_d_T_ = d_d_T - d_epsilon_vec_T
d_d_N_ = d_d_N - d_epsilon_vec_N

In [ ]:
plt.figure(figsize=(16,8))
ylims = plt.ylim()
plt.plot(l_times, dd_epsilon_vec_T, linewidth=5)
plt.autoscale(enable=True, axis='x', tight=True)
plt.ylim(-10, 10)
plt.show()

In [ ]:
sig1_label = r'$\vec p_{C} + \epsilon \hat N$'
sig2_label = r'$\vec p_{M} + r\hat N$'
sig1 = p_C + epsilon_vec
sig2 = p_M+r*N_hats

plot_vec(sig1, sig1_label, sig2, sig2_label)

In [ ]:
sig1_label = r'$\vec p_{C} + \epsilon \hat N$'
sig2_label = r'$\vec p_{M} + r\hat N$'
sig1 = p_C + epsilon_vec - (p_M+r*N_hats)
sig2 = p_C + epsilon_vec - (p_M+r*N_hats)

plot_vec(sig1, sig1_label, sig2, sig2_label)

In [ ]:
plt.figure(figsize=(16,8))
ylims = plt.ylim()
plt.plot(l_times, d_epsilon_vec_N, linewidth=5, 
         label=r'$\hat N$ component of $\ddot{\vec\epsilon}$')
plt.plot(l_times, d_epsilon_vec_T, linewidth=5, 
         label=r'$\hat T$ component of $\ddot{\vec\epsilon}$')
plt.autoscale(enable=True, axis='x', tight=True)
plt.ylim(-1, 1)
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(16,8))
ylims = plt.ylim()
plt.plot(l_times, dd_epsilon_vec_N, linewidth=5, 
         label=r'$\hat N$ component of $\ddot{\vec\epsilon}$')
plt.plot(l_times, dd_epsilon_vec_T, linewidth=5, 
         label=r'$\hat T$ component of $\ddot{\vec\epsilon}$')
plt.autoscale(enable=True, axis='x', tight=True)
plt.ylim(-100, 100)
plt.legend()
plt.show()

### Conclusion
It's alright to neglect $\epsilon$.

##  $d_N$ and separation speed

In [ ]:
separation_speed = logger.data()[log_wrapper.contact_entry_start_idx+9,:]

In [ ]:
lw=5
plt.figure(figsize=(16, 8))
plt.plot(d_times, np.array(debug['d_Ns']), label='$d_N$', linewidth=lw)
plt.axhline(-(constants.FINGER_RADIUS), label='Contact $d_N$ (finger radius)', linestyle='--', color='k', linewidth=lw)
plt.legend()
plt.xlabel("Time (seconds)")
plt.ylabel("Distance (m)")
plt.autoscale(enable=True, axis='x', tight=True)
plt.show()

$d_N$ settles at the contact value, which is what we expect.

In [ ]:
lw=5
plt.figure(figsize=(16, 8))
plt.plot(l_times, separation_speed, label="Separation speed", linewidth=lw)
plt.axhline(0, color='k', linestyle='--', label='Contact', linewidth=lw)
plt.legend()
plt.xlabel("Time (seconds)")
plt.ylabel("Velocity (m/s)")
plt.show()

Separation speed settles at zero, which is what we expect; means we don't break contact.

## Moving contact point
 - [ ] PROGRAMMING: Add arrow heads to contact point plots

In [ ]:
lw=5
plt.figure(figsize=(16, 8))
plt.plot(d_times, debug['d_Ts'], label='$d_T$', linewidth=lw)
plt.axhline(-(0), label='Bounds on $d_T$ (0 and link width)', linestyle='--', color='k', linewidth=lw)
plt.axhline(-(paper.link_width), linestyle='--', color='k', linewidth=lw)
plt.legend()
plt.xlabel("Time (seconds)")
plt.ylabel("Distance (m)")
plt.autoscale(enable=True, axis='x', tight=True)
plt.show()

This plot shows that $d_T$ changes during the simulation an amount which is significant relative to its bounds.

In [ ]:
contact_point = logger.data()[log_wrapper.contact_entry_start_idx+11:log_wrapper.contact_entry_start_idx+14,:]

Sanity check on contact point data:

In [ ]:
plt.figure(figsize=(12,12))

plt.plot(contact_point[1,:], contact_point[2,:], color='k', linestyle='--', linewidth='5')
plt.gca().set_aspect('equal', 'box')
plt.show()

In [ ]:
man_point = logger.data()[log_wrapper.entries_per_body*int(finger_body.index()):
                          log_wrapper.entries_per_body*int(finger_body.index())+3,:]

In [ ]:
plt.figure(figsize=(12,12))
plt.gca().add_patch(plt.Circle((0, 0), constants.FINGER_RADIUS, color='r'))
plt.plot(contact_point[1,:]-man_point[1,:], contact_point[2,:]-man_point[2,:], color='k', linestyle='--', linewidth='5')
plt.gca().set_aspect('equal', 'box')
plt.show()

Shows the contacat point moves around on the surface of the manipulator.

## Friction plots

In [ ]:
slip_speed = logger.data()[log_wrapper.contact_entry_start_idx+10,:]

In [ ]:
t_slip = l_times[np.argmax(slip_speed > v_stiction)]

In [ ]:
lw=5
plt.figure(figsize=(16,12))
plt.plot(l_times, abs(F_FL), label=r"$\left|F_{FL}\right|$", linewidth=lw, color='lightskyblue', zorder=0)
plt.plot(l_times, abs(F_FM), label=r"$\left|F_{ML}\right|$", linewidth=lw, color='b', linestyle='--', zorder=1)
plt.plot(l_times, abs(F_NL)*constants.FRICTION, label=r"$\mu\left|F_{NL}\right|$", linewidth=lw, color='k', linestyle=':')
plt.axvline(t_slip, color='r', label="$t_{slip}$", linewidth=lw)
plt.ylim(-0, 2)
plt.autoscale(enable=True, axis='x', tight=True)
plt.legend()
plt.xlabel("Time")
plt.show()

This plot shows that although we remain within the static friction limit for the majority of the time, at $t_{slip}$ we exceed $v_{stiction}$ and move to dynamic friction.

In [ ]:
lw=5
plt.figure(figsize=(16, 8))
plt.axvline(t_slip, color='r', label="$t_{slip}$", linewidth=lw)
plt.plot(logger.sample_times(), slip_speed, label="slip speed", linewidth=5)
plt.axhline(v_stiction, linestyle="--", color="k", label="$v_{stiction}$", linewidth=5)
plt.xlabel("Time (seconds)")
plt.ylabel("Speed (m/s)")
plt.ylim(0, 2*v_stiction)
plt.autoscale(enable=True, axis='x', tight=True)
plt.legend()
plt.show()

This plot shows what's happening from the slip speed side. Similar to a tire spinning out, as soon as the slip speed exceeds $v_{stiction}$, the positive feedback of lowered friction sends the slip speed quite high.

## Penetration debugging

In [ ]:
lw=5
smooth_idxs = np.arange(800, 4000)
plt.figure(figsize=(16, 8))
plt.plot(l_times, a_MN, linewidth=lw)
plt.plot(l_times[smooth_idxs], a_MN[smooth_idxs], linewidth=lw)
plt.axvline(plant.get_contact_penalty_method_time_scale(), linewidth=lw, color='red')
plt.ylim(-10, 40)
# plt.autoscale(enable=True, axis='x', tight=True)
plt.show()

In [ ]:
finger_shape

In [ ]:
# scene_graph.model_inspector().GetProximityProperties(finger_shape).GetProperty("material", "point_contact_stiffness");

In [ ]:
logger.data().shape

In [ ]:
pen_depth.shape

In [ ]:
lw=5
plt.figure(figsize=(16, 8))
plt.plot(l_times, pen_depth)
plt.ylim(-2,2)
plt.autoscale(enable=True, axis='x', tight=True)
plt.legend()
plt.show()

In [ ]:
# def get_stiffness_and_damping(penetration_allowance) {
penetration_allowance = 0.001
g = 9.81
mass = 0.0

for bi in range(plant.num_bodies()):
    body = plant.get_body(BodyIndex(bi))
    mass = max(mass, body.get_default_mass())

combined_stiffness = mass * g / penetration_allowance
omega = np.sqrt(combined_stiffness / mass)

time_scale = 1.0 / omega

damping_ratio = 1.0
dissipation = damping_ratio * time_scale / penetration_allowance
stiffness = 2 * combined_stiffness

### Pentration into manipulator

In [ ]:
pen_man_point = logger.data()[log_wrapper.contact_entry_start_idx+3:
                                         log_wrapper.contact_entry_start_idx+6]

In [ ]:
np.linalg.norm(pen_man_point[1:3:]-man_point[1:3,:], axis=0).shape

In [ ]:
depth_man = np.linalg.norm(pen_man_point[1:3:]-man_point[1:3,:], axis=0)

In [ ]:
l_times.shape

In [ ]:
plt.figure(figsize=(12,12))
plt.plot(l_times, depth_man, color='k', linestyle='-', linewidth='5')
ylims = plt.ylim()
plt.ylim(0,ylims[-1]*2)
plt.show()

### Penetration into link

In [ ]:
link_point = logger.data()[log_wrapper.entries_per_body*int(paper.link_idxs[-1]):
                           log_wrapper.entries_per_body*int(paper.link_idxs[-1])+3,:]

In [ ]:
pen_link_point = logger.data()[log_wrapper.contact_entry_start_idx+6:
                               log_wrapper.contact_entry_start_idx+9]

In [ ]:
depth_link = np.linalg.norm(pen_link_point[1:3:]-link_point[1:3,:], axis=0)

In [ ]:
plt.figure(figsize=(12,12))
plt.plot(l_times, depth_man, color='k', linestyle='-', linewidth='5')
ylims = plt.ylim()
plt.ylim(0,ylims[-1]*2)
plt.show()

## Accel debugging

In [ ]:
v_man_diff = np.diff(man_point[1:3,:])/np.diff(l_times)
a_man_diff = np.diff(v_man_diff)/np.diff(l_times[:-1])

In [ ]:
lw=5
plt.figure(figsize=(16,16))
for i in range(1,3):
    plt.subplot(2,1,i)
    plt.plot(l_times[:-2][::10], a_man_diff[i-1,:][::10], linewidth=lw)
    plt.plot(l_times, a_man[:,i,0], linewidth=lw)
    plt.ylim(-150, 150)
    plt.legend()
    plt.xlabel("Time")
plt.show()

In [ ]:
v_contact_diff = np.diff(contact_point[1:3,:])/np.diff(l_times)
a_contact_diff = np.diff(v_contact_diff)/np.diff(l_times[:-1])

In [ ]:
a_contact_diff.shape

In [ ]:
lw=5
plt.figure(figsize=(16,16))
for i in range(1,3):
    plt.subplot(2,1,i)
    plt.plot(l_times[:-2][::10], a_contact_diff[i-1,:][::10], linewidth=lw)
    plt.plot(l_times, a_man[:,i,0], linewidth=lw)
    plt.ylim(-150, 150)
    plt.legend()
    plt.xlabel("Time")
plt.show()

## Other $\vec d$ checks

### $\vec d \triangleq \vec p_C - \vec p_E$ math

#### $\dot{\vec d} = \vec v_C - \vec v_E$

In [ ]:
sig1_label = r'$\dot{\vec d}$'
sig2_label = r'$\vec v_C - \vec v_E$'

sig1 = d_d_vec
sig2 = v_C - v_E

plot_vec(sig1, sig1_label, sig2, sig2_label)

#### $\ddot{\vec d} = \vec a_C - \vec a_E$

In [ ]:
sig1_label = r'$\ddot{\vec d}$'
sig2_label = r'$\vec a_C - \vec a_E$'

sig1 = moving_average_vec(dd_d_vec, 10)
sig2 = moving_average_vec(a_C - a_E, 10)

plot_vec(sig1, sig1_label, sig2, sig2_label, finish_plotting=False)
plt.ylim(-1e3, 2e3)
plt.show()

#### How big a difference does it make to use delayed filter?

In [ ]:
sig1_label = r'No delay'
sig2_label = r'Delay'

window = 100
sig1 = moving_average_vec(dd_d_vec, window)

plt.figure(figsize=(16, 8))
lw = 5
plt.plot(l_times, sig1[:,0], label=sig1_label + r' $\hat x$ component', color='pink', linewidth=lw)
plt.plot(l_times, sig1[:,1], label=sig1_label + r' $\hat y$ component', color='lightgreen', linewidth=lw)
plt.plot(l_times, sig1[:,2], label=sig1_label + r' $\hat z$ component', color='lightskyblue', linewidth=lw)

plt.plot(l_times[window:], sig1[window:,0], label=sig2_label + r' $\hat x$ component', color='red', linewidth=lw, linestyle='--')
plt.plot(l_times[window:], sig1[window:,1], label=sig2_label + r' $\hat y$ component', color='green', linewidth=lw, linestyle='--')
plt.plot(l_times[window:], sig1[window:,2], label=sig2_label + r' $\hat z$ component', color='blue', linewidth=lw, linestyle='--')
plt.xlabel("Time (seconds)")
plt.legend()
plt.autoscale(enable=True, axis='x', tight=True)
plt.ylim(-1e3, 2e3)
plt.show()

In [ ]:
np.mean(np.diff(l_times))*100

In [ ]:
sig1_label = r'No delay'
sig2_label = r'Delay'

window = 100
sig1 = moving_average_vec(dd_d_vec, window)

plt.figure(figsize=(16, 8))
lw = 5
plt.plot(l_times, sig1[:,0], label=sig1_label + r' $\hat x$ component', color='pink', linewidth=lw)
plt.plot(l_times, sig1[:,1], label=sig1_label + r' $\hat y$ component', color='lightgreen', linewidth=lw)
plt.plot(l_times, sig1[:,2], label=sig1_label + r' $\hat z$ component', color='lightskyblue', linewidth=lw)

plt.plot(l_times[window:], sig1[window:,0], label=sig2_label + r' $\hat x$ component', color='red', linewidth=lw, linestyle='--')
plt.plot(l_times[window:], sig1[window:,1], label=sig2_label + r' $\hat y$ component', color='green', linewidth=lw, linestyle='--')
plt.plot(l_times[window:], sig1[window:,2], label=sig2_label + r' $\hat z$ component', color='blue', linewidth=lw, linestyle='--')
plt.xlabel("Time (seconds)")
plt.legend()
plt.xlim(0.05, 0.051)
plt.ylim(-1e3, 2e3)
plt.show()

Using averages should be fine.

### $p_C = p_M + r\hat N$

In [ ]:
sig1_label = r'$p_C$'
sig2_label = r'$p_M + r\hat N$'

sig1 = p_C
sig2 = p_M + r*N_hats

plot_vec(sig1, sig1_label, sig2, sig2_label, finish_plotting=False)
# plt.ylim(-1e3, 2e3)
plt.show()

In [ ]:
pen_depth = logger.data()[log_wrapper.contact_entry_start_idx+17]

In [ ]:
lhs = np.linalg.norm(p_C-(p_M + r*N_hats), axis=1)
lhs_label = r'$\left|\left|p_C-\left(p_M+r\hat N\right)\right|\right|$'
rhs = np.expand_dims(pen_depth/2, 1)
rhs_label = "Penetration depth/2"

plot_eq(lhs, lhs_label, rhs, rhs_label)